# Transformación y limpieza - Contratos

In [7]:
# IMPORTS
import pandas as pd
import re, os
import unicodedata

In [8]:
# RUTAS
BASE_PATH = r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES"
DATAPATH = os.path.join(BASE_PATH, "data", "01_filter")
os.makedirs(DATAPATH, exist_ok=True)

In [9]:
# REGISTROS - Seleccionar variables de interés

df = pd.read_excel("../data/00_raw/secop_obras_2019_2025.xlsx")
df = df[[
    # Sobre la entidad
    'nombre_entidad', 'nit_entidad', 'codigo_entidad', 'sector',
    # Sobre el contrato
    'id_contrato', 'estado_contrato','modalidad_de_contratacion', 'anio', 
    'codigo_de_categoria_principal','descripcion_del_proceso', 'fecha_de_firma', 
    # Sobre el proveedor
    'tipodocproveedor', 'documento_proveedor', 'codigo_proveedor', 'proveedor_adjudicado', 
    'origen_de_los_recursos', 'destino_gasto', 'valor_del_contrato', 
    # Sobre el contrato
    'c_digo_bpin', 'urlproceso',
    # Sobre los costos
    'presupuesto_general_de_la_nacion_pgn', 'sistema_general_de_participaciones', 
    'sistema_general_de_regal_as','recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
    'recursos_de_credito', 'recursos_propios'
]]

df = df.apply(lambda col: col.str.lower() if col.dtype == "object" else col)

# Definición de nombres estandar
names = {
    'modalidad_de_contratacion' : 'modalidad_contrato', 
    'codigo_de_categoria_principal': 'codigo_categoria', 
    'descripcion_del_proceso': 'descripcion', 
    'c_digo_bpin': 'codigo_BPIN',
    'presupuesto_general_de_la_nacion_pgn': 'recursos_PGN', 
    'sistema_general_de_participaciones' : 'recursos_SGP', 
    'sistema_general_de_regal_as': 'recursos_SGR',
    'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_': 'recursos_territorio',
    'recursos_de_credito': 'recursos_credito', 
}
df = df.rename(columns=names)

print(f"📍 DataFrame RAW:\n Dimensiones: {df.shape}")

📍 DataFrame RAW:
 Dimensiones: (8400, 26)


In [15]:
# FILTRO - Estado del contrato

# Se seleccionan solo los contratos en estado de ejecución o terminados.
ESTADOS_CORE = [
    "terminado",
    "modificado",
    "en ejecución",
    "cerrado"
]
df["estado_contrato"] = (df["estado_contrato"].str.strip().str.lower())
filter = df[df["estado_contrato"].isin(ESTADOS_CORE)].copy()

tabla_1 = (filter["estado_contrato"].value_counts(dropna=False).rename("conteo").reset_index())
tabla_1["porcentaje"] = (tabla_1["conteo"] / tabla_1["conteo"].sum() * 100).round(2)
tabla_1.columns = ["estado_contrato", "conteo", "porcentaje"]
display(tabla_1)

print("\nContratos core:", filter.shape[0])
print("Porcentaje retenido:", round(filter.shape[0] / df.shape[0] * 100, 2), "%")


,estado_contrato,conteo,porcentaje
0,terminado,3097,39.90
1,modificado,2764,35.61
2,en ejecución,1024,13.19
3,cerrado,876,11.29



Contratos core: 7761
Porcentaje retenido: 92.39 %


In [16]:
# MAPEO - Familia UNSPSC 

# Categoría de contrato según la familia del código (Descripción de qué se hace, deifine la naturaleza técnica)
MAP_FAMILIA_UNSPSC = {
    "7210": "Mantenimiento y reparaciones",
    "7211": "Edificación residencial",
    "7212": "Edificación no residencial",
    "7214": "Infraestructura pesada",
    "7215": "Infraestructura especializada"
}
filter["codigo_familia_UNSPSC"] = (filter["codigo_categoria"].str.extract(r"v1\.(72\d{2})"))
filter["nombre_familia_UNSPSC"] = (filter["codigo_familia_UNSPSC"].map(MAP_FAMILIA_UNSPSC))

tabla_2 = (filter["nombre_familia_UNSPSC"].value_counts(dropna=False).rename("conteo").reset_index())
tabla_2["porcentaje"] = (tabla_2["conteo"] / tabla_2["conteo"].sum() * 100).round(2)
tabla_2.columns = ["nombre_familia_UNSPSC", "conteo", "porcentaje"]
display(tabla_2)


,nombre_familia_UNSPSC,conteo,porcentaje
0,Mantenimiento y reparaciones,3641,46.91
1,Infraestructura pesada,2384,30.72
2,Infraestructura especializada,1015,13.08
3,Edificación no residencial,650,8.38
4,Edificación residencial,71,0.91


In [12]:
# ESTANDARIZACIÓN - Texto descriptivo del proceso de contratación

def estandarizar_descripcion(texto):
    if pd.isna(texto):
        return texto
    original = texto
    # 1. minusculas
    texto = texto.lower()
    # 2. eliminar signos de puntuacion basicos
    texto = re.sub(r'[.,;:]', '', texto)
    # 3. eliminar comillas y caracteres especiales comunes
    texto = re.sub(r'[\"“”\'’]', '', texto)
    # 4. normalizar unicode (quita tildes pero conserva ñ)
    texto = unicodedata.normalize("NFD", texto)
    texto = "".join(ch for ch in texto if unicodedata.category(ch) != "Mn" or ch == "ñ")
    # 5. eliminar números
    texto = re.sub(r'\d+', '', texto)
    # 6. eliminar caracteres no alfabéticos (excepto espacios y ñ)
    texto = re.sub(r'[^a-zñ\s]', ' ', texto)
    # 7. normalizar espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

filter['descripcion_std'] = filter['descripcion'].apply(estandarizar_descripcion)

In [13]:
# Correccion de digitación
reemplazos = {
    "manteniendo": "mantenimiento",
    "manteniiento": "mantenimiento",
    "matenimiento": "mantenimiento",
    "construcion": 'construccion',
    'construicion': 'construccion',
    'construiccion': 'construccion',
}
filter["descripcion_std"] = filter["descripcion_std"].replace(reemplazos, regex=True)

In [14]:
# GUARDAR - Datos con macro filtro
output_file = os.path.join(DATAPATH, "SECOP_filtro_contrato.xlsx")
filter.to_excel(output_file, index=False)

________